# Pasos a seguir

Ejecutar código: Menú Entorno de ejecución / Ejecutar todo






#Instalacion de paqueterías

In [ ]:
#Conectarse al SFTP
import ftplib
import gzip
import zipfile
import shutil
import pandas as pd

import os
from os import listdir
import json

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.cloud import storage

gcs_service = build('storage', 'v1',)
bucket_name="vb-infarecargamanual001"

#Credenciales de la cuenta de servicio


In [ ]:
credentials_dict = {
  "type": "service_account",
  "project_id": "vb-datalake-01-prd",
  "private_key_id": "EXAMPLE",
  "private_key": "EXAMPLE",
  "client_email": "EXAMPLE",
  "client_id": "EXAMPLE",
  "auth_uri": "EXAMPLE",
  "token_uri": "EXAMPLE",
  "auth_provider_x509_cert_url": "EXAMPLE",
  "client_x509_cert_url": "EXAMPLE"
}

with open("credentials.json", "w") as outfile:
    json.dump(credentials_dict, outfile)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/credentials.json"    

#Conexion al SFTP

In [ ]:
#Conectarse al SFTP

#Open ftp connection
ftp = ftplib.FTP(host='ftp.infare.net',user='user', passwd='passwd',timeout=10800)

#List the files in the current directory
print ("File List:")
files = ftp.dir()
print (files)

File List:
drwxrwxrwx   1 user     group           0 Jun 30 22:23 AirRM
drwxrwxrwx   1 user     group           0 Jun 22 09:17 year=2019
None


In [ ]:
#Archivos en SFTP
path="/AirRM/Archive"
ftp.cwd(path)

files = []
files=[filename for filename in ftp.nlst(path) if filename.endswith('.zip')]

#Se obtienen los archivos que no estan cargados en Cloud storage

In [ ]:
#Archivos en cloud storage
from google.cloud import storage

storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name)
CloudStorageFiles= [blob.name.replace(".csv", ".zip") for blob in blobs]

In [ ]:
#Archivos pendientes
ArchivosPendientes = [file for file in files if file not in CloudStorageFiles] 
ArchivosPendientes

['VB_INF_300620221420.zip',
 'VB_INF_300620221440.zip',
 'VB_INF_300620221500.zip',
 'VB_INF_300620221520.zip']

In [ ]:
directory=os.getcwd()
directory

'/content'

# Ejecución de data transfer y lista de archivos a subir a Cloud storage

In [ ]:
#CARGAR LOS ARCHIVOS AL DIRECTORIO
for file in ArchivosPendientes:
  localfile = open(file, 'wb')
  ftp.retrbinary('RETR ' + file, localfile.write, 1024)
  localfile.close()

#ENLISTAR LOS ARCHIVOS .zip
zipfiles = [f for f in listdir(directory) if f.endswith('.zip')]
print("El numero de archivos 'zip' cargados es: {}".format(len(zipfiles))) 

#Desempaquetarlos y borrar archivos .zip
for zip_file in zipfiles:
  with zipfile.ZipFile(directory + "/"+ zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory)
    os.remove(zip_file)

csvtoupload = [f for f in listdir(directory) if f.endswith('.csv')]
print("El numero de archivos '.csv' desempaquetados es: {}".format(len(csvtoupload)))
print("\nArchivos a subir:\n")
print(csvtoupload)


El numero de archivos 'zip' cargados es: 6
El numero de archivos '.csv' desempaquetados es: 6

Archivos a subir:

['VB_INF_290620221520.csv', 'VB_INF_290620221500.csv', 'VB_INF_290620221440.csv', 'VB_INF_290620221600.csv', 'VB_INF_290620221620.csv', 'VB_INF_290620221540.csv']


# Subir archivos a Cloud Storage

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

gcs_service = build('storage', 'v1',)
bucket_name="vb-infarecargamanual001"

for f in csvtoupload:
    name= f
    media = MediaFileUpload(directory+"/"+ f,resumable=True)
    request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name= f,
                                       media_body=media)
    response = None
    while response is None:
       _, response = request.next_chunk()
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)

    print('{} Upload complete'.format(f))

VB_INF_290620221420.csv Upload complete
VB_INF_290620221340.csv Upload complete
VB_INF_290620221400.csv Upload complete
VB_INF_290620221320.csv Upload complete


#Crear tabla en Bigquery con archivos de Cloud Storage

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "vb-datalake-01-prd.Viva_Infare.Captures"

#Borrar tabla existente
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))

#Configurar tabla 
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("observation_date", "DATE"),
        bigquery.SchemaField("observation_time", "STRING"),
        bigquery.SchemaField("pos", "STRING"),
        bigquery.SchemaField("origin", "STRING"),
        bigquery.SchemaField("destination", "STRING"),
        bigquery.SchemaField("is_one_way", "INTEGER"),
        bigquery.SchemaField("outbound_travel_stop_over", "STRING"),
        bigquery.SchemaField("inbound_travel_stop_over", "STRING"),
        bigquery.SchemaField("carrier", "STRING"),
        bigquery.SchemaField("outbound_flight_no", "STRING"),
        bigquery.SchemaField("inbound_flight_no", "STRING"),
        bigquery.SchemaField("outbound_departure_date", "TIMESTAMP"),
        bigquery.SchemaField("outbound_arrival_date", "TIMESTAMP"),
        bigquery.SchemaField("inbound_departure_date", "TIMESTAMP"),
        bigquery.SchemaField("inbound_arrival_date", "TIMESTAMP"),
        bigquery.SchemaField("outbound_fare_basis", "STRING"),
        bigquery.SchemaField("inbound_fare_basis", "STRING"),
        bigquery.SchemaField("outbound_booking_class", "STRING"),
        bigquery.SchemaField("inbound_booking_class", "STRING"),
        bigquery.SchemaField("price_exc", "FLOAT"),
        bigquery.SchemaField("price_inc", "FLOAT"),
        bigquery.SchemaField("tax", "FLOAT"),
        bigquery.SchemaField("currency", "STRING"),
        bigquery.SchemaField("source", "STRING"),
        bigquery.SchemaField("price_outbound", "FLOAT"),
        bigquery.SchemaField("price_inbound", "FLOAT"),
        bigquery.SchemaField("is_tax_inc_outin", "INTEGER")    
    ],
    skip_leading_rows=1,
)

#Anterior URI
#uri = "gs://vb-infarecargamanual001/VB_INF_*.csv"

uri = "gs://vb-infarecargamanual001/*.csv"


load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Wait for the job to complete.

table = client.get_table(table_id)
print("Loaded {} rows to table {}".format(table.num_rows, table_id))

Deleted table 'vb-datalake-01-prd.Viva_Infare.Captures'.
Loaded 42984464 rows to table vb-datalake-01-prd.Viva_Infare.Captures


#Limpiar directorio de archivos y cerrar conexión con FTP

In [ ]:
onlyfiles = [f for f in listdir(directory) if f.endswith('.csv')]

for f in onlyfiles:
  os.remove(f)

os.remove("/content/credentials.json")
#Cerrar conexion sftp
ftp.quit()  

'221 Goodbye.'